# Get FWS TESS information and cache it in tircache

This notebook builds on the previous notebook that caches from ITIS. This one uses the same basic methods to get information from the US Fish and Wildlife Service's Ecological Conservation Online System (ECOS), Threatened and Endangered Species System (TESS) on Federal listing status under the Endangered Species Act. This information is cached in the tircache.

In [60]:
import requests,datetime,configparser
from IPython.display import display
from lxml import etree
from io import StringIO, BytesIO

In [61]:
# Set defaults
targetData = "sgcn"
dt = datetime.datetime.utcnow().isoformat()
tessSpeciesQueryByTSNBaseURL = "https://ecos.fws.gov/ecp0/TessQuery?request=query&xquery=/SPECIES_DETAIL[TSN="

In [62]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'/Users/sky/Documents/configs/stuff.py'))

In [63]:
# Build base URL with API key using input the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [64]:
# Basic function to insert subject ID, property, and value into tircache
def insertTupleInTirCache(subjectid,prop,value):
    # Build query string
    insertSQL = "INSERT INTO tircache (subjectid,property,value) VALUES ('"+subjectid+"','"+prop+"','"+value+"')"
    # Execute query
    r = requests.get(getBaseURL()+"&q="+insertSQL)

In [65]:
# Retrieve target data (only SGCN ITIS IDs at this point)
def getSubjectIDs(targetData):
    if targetData == 'sgcn':
        # Build query string to retrieve data
        targetDataSQL = "SELECT DISTINCT sgcn.taxonomicauthorityid_accepted AS subjectid \
            FROM sgcn \
            WHERE sgcn.taxonomicauthorityid_accepted LIKE '%itis%' \
            AND sgcn.taxonomicauthorityid_accepted NOT IN \
            (SELECT tircache.subjectid FROM tircache WHERE tircache.subjectid LIKE '%itis%' \
            AND tircache.property LIKE 'TESS:RecordCheck%') \
            LIMIT 4000"
        # Get Data
        targetData = requests.get(getBaseURL()+"&q="+targetDataSQL, verify='/Users/sky/cacert.pem').json()
        return targetData

In [66]:
# Extract the TSN from the longer "URI-like" string we used for ITIS in the SGCN data
def extractTSN(subjectid):
    tsn = subjectid.replace('http://services.itis.gov/?q=tsn:','')
    return tsn

In [67]:
# Loop through the target data, extract Subject IDs (only ITIS at this point), retrieve data from service, 
# and insert results in tircache
def getTESSInfoFromTSN(targetData):
    for feature in targetData['features']:
        # Set the subjectid for query and recording
        strSubjectId = feature['properties']['subjectid']
        
        # Set the URL path to get the common names from TSN via one of the ITIS JSON service end points, get the response JSON, and pull out the common names structure
        tessQueryURL = tessSpeciesQueryByTSNBaseURL+extractTSN(strSubjectId)+"]"
        tessData = requests.get(tessQueryURL)
        if tessData.text.find('<results/>') > 0:
            insertTupleInTirCache(strSubjectId,"TESS:RecordCheck:NegativeResponse",dt)
        else:
            try:
                insertTupleInTirCache(strSubjectId,"TESS:RecordCheck:PositiveResponse",dt)
                rawXML = tessData.text.replace('<?xml version="1.0" encoding="iso-8859-1"?>', '')
                f = StringIO(rawXML)
                tree = etree.parse(f)
                insertTupleInTirCache(strSubjectId,"TESS:entityId",tree.xpath('/results/SPECIES_DETAIL/ENTITY_ID')[0].text)
                insertTupleInTirCache(strSubjectId,"TESS:SpeciesCode",tree.xpath('/results/SPECIES_DETAIL/SPCODE')[0].text)
                insertTupleInTirCache(strSubjectId,"TESS:CommonName",tree.xpath('/results/SPECIES_DETAIL/COMNAME')[0].text)
                insertTupleInTirCache(strSubjectId,"TESS:PopulationDescription",tree.xpath('/results/SPECIES_DETAIL/POP_DESC')[0].text)
                insertTupleInTirCache(strSubjectId,"TESS:Status",tree.xpath('/results/SPECIES_DETAIL/STATUS')[0].text)
                insertTupleInTirCache(strSubjectId,"TESS:StatusText",tree.xpath('/results/SPECIES_DETAIL/STATUS_TEXT')[0].text)
                rListingDate = tree.xpath('/results/SPECIES_DETAIL/LISTING_DATE')
                if len(rListingDate) > 0:
                    insertTupleInTirCache(strSubjectId,"TESS:ListingDate",rListingDate[0].text)
            except:
                print ("Problem with "+tessQueryURL)

In [68]:
# Run the process by firing functions
getTESSInfoFromTSN(getSubjectIDs("sgcn"))